In [1]:
#These are pip install commands, which will download all the libraries and tools you'll need for this project.
#Once you've installed these, you shouldn't have to keep re-running this cell.
!pip install kaggle
!pip install pandas
!pip install numpy
!pip install os
!pip install torch
!pip install torchvision
!pip install torch.optim
!pip install torch.utils.data
!pip install torchvision.transforms
!pip install PIL

KeyboardInterrupt: 

In [ ]:
#Connect to your Google account that has your Kaggle API in your Drive.
!pip install kaggle
from google.colab import drive
drive.mount('/content/drive')
#These are the imports you will need for this project. If you find a tool from another library you want to use, you can import here.
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import time
import copy

In [ ]:
#Set up your Kaggle account and add the JSON file into your Google Drive.
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/kaggle.json"
#We're downloading our modified brand dataset and unzipping it into our directory.
!kaggle datasets download -d aditikashi/stanford-car-dataset-brands-only-csv-files
!unzip stanford-car-dataset-brands-only-csv-files.zip

Dataset URL: https://www.kaggle.com/datasets/aditikashi/stanford-car-dataset-brands-only-csv-files
License(s): apache-2.0
stanford-car-dataset-brands-only-csv-files.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  stanford-car-dataset-brands-only-csv-files.zip
replace cardata/cars_annos.mat? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace cardata/cars_test/cars_test/00001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace cardata/cars_test/cars_test/00002.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace cardata/cars_test/cars_test/00003.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#This line specify the paths to the CSV file and the image folder. Do NOT edit this, since it matches the imported dataset.
PATH = "cardata/"

#Create two pandas DataFrames that hold the info in the CSV files.
train_df = pd.read_csv(f'{PATH}train.csv')
test_df = pd.read_csv(f'{PATH}test.csv')

# Let's inspect the data!
print("Training Dataset:\n", train_df.head())
print("Testing Dataset:\n", test_df.head())

# Step 2: Check the distribution of car brands, a.k.a how many images of each brand there are.
print("\nCar brand distribution in training dataset:\n", train_df['label'].value_counts())
print("\nCar brand distribution in testing dataset:\n", test_df['label'].value_counts())

# Step 3: Take a random sampling of data. The reason we do this is because Jupyter Notebook can't efficiently run all 8000 training and 8000 testing images, so we have to
#have a smaller data set that can be run here. Don't worry about the semantics of this code, but all it's doing is taking 3000 sample images from each dataset. You can
#change the number 3000 to a different number if you'd like to test it out. We do this process for both training and testing data.
train_sampled_df = train_df.groupby('label').apply(lambda x: x.sample(min(len(x), 3000 // train_df['label'].nunique()))).reset_index(drop=True)
test_sampled_df = test_df.groupby('label').apply(lambda x: x.sample(min(len(x), 3000 // test_df['label'].nunique()))).reset_index(drop=True)

# Step 4: We can save these new datasets to new CSV files, new_train and new_test. The reason we made new files is in case we want to reaccess all of the data at any point.
#For model development, you'll be using these new CSV files.
train_sampled_df.to_csv('/content/cardata/new_train.csv', index=False)
test_sampled_df.to_csv('/content/cardata/new_test.csv', index=False)

#Let's inspect the size of the new datasets!
print("New Training Dataset shape:", train_sampled_df.shape)
print("New Testing Dataset shape:", test_sampled_df.shape)

#TO-DO: Move your new_train and new_test files into the cardata folder.


Training Dataset:
    imagename    label
0  00001.jpg     Audi
1  00002.jpg    Acura
2  00003.jpg    Dodge
3  00004.jpg  Hyundai
4  00005.jpg     Ford
Testing Dataset:
    imagename    label
0  00001.jpg   Suzuki
1  00002.jpg  Ferrari
2  00003.jpg     Jeep
3  00004.jpg   Toyota
4  00005.jpg    Tesla

Car brand distribution in training dataset:
 label
Chevrolet        905
Dodge            630
Audi             589
BMW              531
Ford             521
Hyundai          438
Mercedes-Benz    261
Chrysler         260
Acura            242
GMC              238
Bentley          238
Jeep             220
Nissan           171
Toyota           168
Suzuki           167
Ferrari          164
Honda            161
Lamborghini      161
Buick            158
Aston            157
Volkswagen       132
Volvo            131
Cadillac         129
Rolls-Royce      114
Spyker            88
Land              86
HUMMER            83
Bugatti           77
Infiniti          67
FIAT              62
Mitsubishi       

<ipython-input-6-fa5f843b8eac>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_sampled_df = train_df.groupby('label').apply(lambda x: x.sample(min(len(x), 3000 // train_df['label'].nunique()))).reset_index(drop=True)
<ipython-input-6-fa5f843b8eac>:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_sampled_df = test_df.groupby('label').apply(lambda x: x.sample(min(len(x), 3000 // test_df['label']

In [ ]:
#Okay, now it's time to work on our model! We've given you some of the basic code and function structure you'll need.
#This first section of code essentially reads, writes, and accesses our data. Don't worry about the functionality.
class CustomCarDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        #The three values below this comment are important!!
        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        #Get a list of all the unique car brands for the classes.
        self.classes = self.labels_df['label'].unique().tolist()
        #Maps the labels to the indices.
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        #Get the image file name and label from the dataframe.
        img_name = os.path.join(self.img_dir, self.labels_df.iloc[idx, 0])
        #Open the image and convert to RGB.
        image = Image.open(img_name).convert('RGB')

        #Get the corresponding label and map it to the index
        label = self.class_to_idx[self.labels_df.iloc[idx, 1]]

        #Apply the transformations you created.
        if self.transform:
            image = self.transform(image)

        return image, label

#This is where you'll add your image augmentation. Refer back to the image augmentation notebook for some guidance.
#One function you'll definitely need is transforms.CenterCrop(). This is because the ResNet-50 model only uses images
#with inputs of size 244 by 244 (hint!). Another function you'll need is transforms.ToTensor(), since Tensorflow will only process data
#that is in tensor form. Finally, transforms.Normalize() is a way to standardize the images using ImageNet, which is typical
#for any models that were pre-trained using ImageNet, like ResNet50! Essentially, ResNet was originally trained on the images
#in ImageNet, so we adjust the range of our pixel values to match the range used in the ImageNet dataset.

#We've given you some of the functions you'll need, so it's your job to go through documentation and test out some code.

#TO-DO: Add the necessary functions we mentioned, and experiment with more image augmentation techniques that change more aspects of the images.
#Again, refer back to the image augmentation notebook for this. Try to see what combinations can increase your accuracy.
data_transforms = #TO-DO

#These are file paths for the CSV and the image directories.
train_img_dir = '/content/cardata/cars_train/cars_train'
train_csv = '/content/cardata/new_train.csv'

test_img_dir = '/content/cardata/cars_test/cars_test'
test_csv = '/content/cardata/new_test.csv'

#TO-DO: Create Dataset() instances here for the training and testing. Look at the def_init function at the very
#beginning that we have provided for you. Think about how you could call this class (CustomCardataset()) and what input values
#you might need to use. Hint: since this is a class, it won't be exactly like calling a function
train_dataset = CustomCarDataset(csv_file=train_csv, img_dir=train_img_dir, transform=data_transforms)
test_dataset = CustomCarDataset(csv_file=test_csv, img_dir=test_img_dir, transform=data_transforms)

#TO-DO: Create DataLoader() instances here for the training and testing. Documentation is your friend! Search for
#DataLoader documentation, and experiment with the parameters you give the function.
train_loader = #TO-DO
test_loader = #TO-DO

# Prepare dataloaders and dataset sizes for the training and testing phases.
dataloaders = {
    'train': train_loader,
    'test': test_loader,
}

dataset_sizes = {
    'train': len(train_dataset),
    'test': len(test_dataset),
}

#Get the class names (car brands) and the number of car brands.
class_names = train_dataset.classes
num_classes = len(class_names)

#Check if we can use torch.cuda for fast implementaiton or CPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Here's where you'll be implementing your model. Again, documentation, Google, and ChatGPT is your friend.
#We'll leave in variable names so you have a general idea of what to follow and so that the code flows with the
#pre-written aspects.

#Here is where you set the model to ResNet-50. Look at some of the parameters you might need.
model_ft = #TO-DO
model_ft = model_ft.to(device)

#Here is where you will add some of the features of the model, like the loss function, optimizer, and learning
#rate scheduler. You can use whatever features you'd like, and keep experimenting with different hyperparameters to
#see what works best. Once again, look at documentation!
num_ftrs = model_ft.fc.in_features
criterion = #TO-DO
optimizer_ft = #TO-DO
exp_lr_scheduler = #TO-DO

#Modify the fully connected layer to match the number of car brands (hint: where did we get the number of car brands?).
model_ft.fc = #TO-DO

#This section of the code creates the training and testing pipeline. While this is an important part of the computer vision
#pipeline, it mainly involves iterating over data and finding accuracy, which are not as critical to your fundamental
#understanding of computer vision as a whole. We've given you the code that will iterate through your model. Feel free to change
#or add anything you'd like (hint: you can look into things like freezing a layer, adding a layer, etc).

#TO-DO: Experiment as much as you'd like with adding layers, freezing layers, anything you'd like to try.
#We'll go over some features you can add to make your model better in future sessions, but feel free to try stuff out yourself.
def make_model(model, criterion, optimizer, scheduler, num_epochs=25):

    since = time.time()

    #Save the best model and best accuracy.
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        #Each epoch has a training and validation phase.
        for phase in ['train', 'test']:
            if phase == 'train':
                #Set the model to training mode.
                model.train()
            else:
                #Set the model to testing mode.
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            #Iterate over all the data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                #Zero the parameter gradients.
                optimizer.zero_grad()

                #TO-DO: This is a place where you could add more features, like cross-validation, freezing, etc.

                #Forward pass: run through the model forwards.
                #Only track the history of the model if it's in train mode.
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    #Backward pass + optimize: this only happens while in train mode.
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                #Calculate running loss function and the number of correct guesses.
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            #Make a deep copy of the best model and update accuracy.
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    #Check how much time training took.
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best test Acc: {best_acc:.4f}')

    #Load the best model weights for the next epoch.
    model.load_state_dict(best_model_wts)
    return model

#Write this function that calls the training function with all its parameters.
model_ft = #TO-DO

#Save the best model.
torch.save(model_ft.state_dict(), 'car_brand_model.pth')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvisio

Epoch 0/24
----------


KeyboardInterrupt: 